In [1]:
import torch
from Segmentation.spvcnn import *
from torchsparse import SparseTensor
from torchsparse.utils.quantize import sparse_quantize
from torchsparse.utils.collate import sparse_collate
from ConvBKI.ConvBKI import *
from Propagation.mapping_utils import *
from BKINet import *

In [2]:
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud
from nuscenes.utils.data_classes import LidarSegPointCloud
from nuscenes.utils.data_io import load_bin_file
from pyquaternion import Quaternion
from nuscenes.utils.geometry_utils import transform_matrix
data_dir = '/home/tigeriv/Data/nuscenes_mini'
nusc = NuScenes(version='v1.0-mini', dataroot=data_dir, verbose=True)

import plotly
import plotly.graph_objs as go

Loading NuScenes tables for version v1.0-mini...
Loading nuScenes-lidarseg...
32 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
404 lidarseg,
Done loading in 0.262 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [3]:
dev = 'cuda' if torch.cuda.is_available() else 'cpu'

dtype=torch.float32

voxel_sizes = torch.tensor([0.25, 0.25, 0.25]).to(dev)

grid_size = torch.tensor([200, 200, 20]).to(dev)
min_bound = torch.tensor([-25., -25., -3.5]).to(dev)
max_bound = torch.tensor([25., 25., 1.5]).to(dev)

num_classes = 13

f = 5

seg_path = "/home/tigeriv/Code/NuScenesConvBKI/spvnas/runs/Downsampled/checkpoints/max-iou-test.pt"

COLOR_MAP = np.array(['#f59664', '#f5e664', '#963c1e', '#b41e50', '#ff0000',
                      '#1e1eff', '#c828ff', '#5a1e96', '#ff00ff', '#ff96ff',
                      '#4b004b', '#4b00af', '#00c8ff', '#3278ff', '#00af00'])

In [4]:
# Segmentation network
seg_net = SPVCNN(
    num_classes=num_classes,
    cr=0.5,
    pres=0.05,
    vres=0.05).to(dev)

seg_net.load_state_dict(torch.load(seg_path)['model'])

<All keys matched successfully>

In [5]:
# Propagation network
prop_net = TransformWorldStatic(voxel_sizes)

In [6]:
# ConvBKI network
bki_layer = ConvBKI(grid_size, min_bound, max_bound, 
                    filter_size=f, num_classes=num_classes, device=dev)

/home/tigeriv/.local/lib/python3.8/site-packages/torch/functional.py:1069: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.cartesian_prod(tensors)  # type: ignore[attr-defined]


In [7]:
# End to End Network
e2e_net = BKINet(seg_net, bki_layer, prop_net, grid_size, device=dev, num_classes=num_classes)

In [8]:
def get_lidar(curr_sample):
    # Load lidar pc
    lidar_data = nusc.get('sample_data', curr_sample['data']['LIDAR_TOP'])
    ego_pose = nusc.get('ego_pose', lidar_data['ego_pose_token'])
    lidar_pose = transform_matrix(translation=ego_pose['translation'], rotation=Quaternion(ego_pose['rotation']))
    
    lidar_fpath = os.path.join(data_dir, lidar_data['filename'])
    label_data = nusc.get('lidarseg', curr_sample['data']['LIDAR_TOP'])
    label_fpath = os.path.join(data_dir, label_data['filename'])

    labeled_pc = LidarSegPointCloud(points_path=lidar_fpath, labels_path=label_fpath)
    return lidar_pose, labeled_pc
    

def generate_seg_in(lidar):
    # Create input data
    coords = np.round(lidar[:, :3] / 0.05)
    coords -= coords.min(0, keepdims=1)
    feats = lidar
    # Filter out duplicate points
    coords, indices, inverse = sparse_quantize(coords, return_index=True, return_inverse=True)
    coords = torch.tensor(coords, dtype=torch.int)
    feats = torch.tensor(feats[indices], dtype=torch.float)

    inputs = SparseTensor(coords=coords, feats=feats)
    inputs = sparse_collate([inputs]).cuda()
    return inputs, inverse

In [9]:
# Test by loading weights for segmentation, freezing everything
# and running on nuScenes
curr_scene = nusc.scene[2]

sample_token = curr_scene['first_sample_token']
curr_sample = nusc.get('sample', sample_token)

In [17]:
curr_sample = nusc.get('sample', curr_sample['next'])

lidar_pose, labeled_pc = get_lidar(curr_sample)
lidar = labeled_pc.points
seg_input, inv = generate_seg_in(lidar)

In [18]:
labeled_points = seg_net(seg_input)[inv]
outputs = labeled_points.argmax(1).cpu().numpy()

In [19]:
e2e_net.requires_grad = False
lidar[:, 3] = 1
input_data = [torch.tensor(lidar_pose).to(dev).type(dtype), torch.tensor(lidar).to(dev).type(dtype),
              seg_input, torch.tensor(inv).to(dev)]
e2e_net(input_data)

In [13]:
# Plot lidar predictions

%matplotlib inline

def configure_plotly_browser_state():
    import IPython
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

def plot_lidar_preds(lidar):
    trace = go.Scatter3d(
        x=lidar[:, 0],
        y=lidar[:, 1],
        z=lidar[:, 2],
        mode='markers',
        marker={
            'size': 1,
            'opacity': 0.8,
            'color': COLOR_MAP[outputs].tolist(),
        }
    )

    configure_plotly_browser_state()
    plotly.offline.init_notebook_mode(connected=False)

    layout = go.Layout(
        margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
        scene=dict(aspectmode='manual', aspectratio=dict(x=1, y=1, z=0.2))
    )

    plotly.offline.iplot(go.Figure(data=[trace], layout=layout))